In [1]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import parse_kraken_orderbook
from hydradx.model.amm.arbitrage_agent import get_arb_swaps, calculate_profit, execute_arb
from hydradx.model.amm.centralized_market import CentralizedMarket, OrderBook
from hydradxapi import HydraDX
from pprint import pprint
import requests



import json, os


In [2]:

# TODO move to config file
arb_list = [
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 10), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 23), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 2), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 7), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 18), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 21), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 22), 'order_book': ('HDX','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 10), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 23), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 2), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 7), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 18), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 21), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 22), 'order_book': ('DOT','USD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 10), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 23), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 2), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 7), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 18), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 21), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 22), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 10), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 23), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 2), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 7), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 18), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 21), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 22), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('DOT', 'WETH'), 'tkn_ids': (5, 4), 'order_book': ('DOT','ETH')},
    {'tkns': ('DOT', 'WETH'), 'tkn_ids': (5, 20), 'order_book': ('DOT','ETH')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 10), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 23), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 2), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 7), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 18), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 21), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 22), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 10), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 23), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 2), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 7), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 18), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 21), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 22), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 10), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 23), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 2), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 7), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 18), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 21), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 22), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('DOT', 'WBTC'), 'tkn_ids': (5, 3), 'order_book': ('DOT','XBT')},
    {'tkns': ('DOT', 'WBTC'), 'tkn_ids': (5, 19), 'order_book': ('DOT','XBT')},
    {'tkns': ('DOT', 'IBTC'), 'tkn_ids': (5, 11), 'order_book': ('DOT','XBT')},
    # {'tkns': ('WETH', 'WBTC'), 'tkn_ids': (4, 3), 'order_book': ('XETH','XXBT')},
    {'tkns': ('WETH', 'WBTC'), 'tkn_ids': (4, 19), 'order_book': ('XETH','XXBT')},
    {'tkns': ('WETH', 'IBTC'), 'tkn_ids': (4, 11), 'order_book': ('XETH','XXBT')},
    # {'tkns': ('WETH', 'WBTC'), 'tkn_ids': (20, 3), 'order_book': ('XETH','XXBT')},
    {'tkns': ('WETH', 'WBTC'), 'tkn_ids': (20, 19), 'order_book': ('XETH','XXBT')},
    {'tkns': ('WETH', 'IBTC'), 'tkn_ids': (20, 11), 'order_book': ('XETH','XXBT')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 10), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 23), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 2), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 7), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 18), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 21), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 22), 'order_book': ('ASTR','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 10), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 23), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 2), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 7), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 18), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 21), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 22), 'order_book': ('CFG','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 10), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 23), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 2), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 7), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 18), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 21), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 22), 'order_book': ('BNC','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 10), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 23), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 2), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 7), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 18), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 21), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 22), 'order_book': ('GLMR','USD')},
]

order_books = {}
for arb_cfg in arb_list:
    tkn_pair = arb_cfg['order_book']
    if tkn_pair not in order_books:
        resp = requests.get('https://api.kraken.com/0/public/Depth?pair=' + tkn_pair[0] + tkn_pair[1])
        y = resp.json()
        order_books[tkn_pair] = y['result'][tkn_pair[0] + tkn_pair[1]]
    


In [3]:
def get_unique_name(ls: list[str], name: str) -> str:
    if name not in ls:
        return name
    else:
        c = 1
        while name + str(c).zfill(3) in ls:
            c += 1
        return name + str(c).zfill(3)

In [4]:
RPC="wss://hydradx-rpc.dwellir.com"
N = 24
metadata = []
with HydraDX(RPC) as chain:
    for i in range(N):
        try:
            md = chain.api.registry.asset_metadata(i)
            metadata.append({'symbol': md.symbol, 'id': md.asset_id, 'decimals': md.decimals},)
        except:
            continue

    asset_list = []
    fees = {}
    tokens = {}
    asset_map = {}
    
    for i in range(len(metadata)):
        try:
            state = chain.api.omnipool.asset_state(metadata[i]['id'])
        except:
            continue
            
        fee = chain.api.fees.asset_fees(metadata[i]['id'])
    
        tkn = get_unique_name(asset_list, metadata[i]['symbol'])
        asset_list.append(tkn)
        asset_map[i] = tkn
        tokens[tkn] = {
            'liquidity': state.reserve / 10**metadata[i]['decimals'],
            'LRNA': state.hub_reserve / 10**12
        }
        fees[tkn] = {"asset_fee": fee.asset_fee / 100, "protocol_fee": fee.protocol_fee / 100}
            

pprint(tokens)


{'ASTR': {'LRNA': 9082.738326360857, 'liquidity': 4613307.689932842},
 'BNC': {'LRNA': 6960.530069554551, 'liquidity': 723868.8934341213},
 'CFG': {'LRNA': 8851.608850641, 'liquidity': 602207.3975931228},
 'DAI': {'LRNA': 13110.676857870463, 'liquidity': 332743.9021445043},
 'DAI001': {'LRNA': 31824.663044315806, 'liquidity': 799910.0687595578},
 'DOT': {'LRNA': 70081.12991599753, 'liquidity': 429429.3214752538},
 'GLMR': {'LRNA': 5249.607197601644, 'liquidity': 677608.617803954},
 'HDX': {'LRNA': 19546.514674768685, 'liquidity': 117711721.7149292},
 'USDT': {'LRNA': 85426.64934008372, 'liquidity': 2149511.795668},
 'WBTC': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WBTC001': {'LRNA': 41825.898662570326, 'liquidity': 31.07199565},
 'WETH': {'LRNA': 21611.81073019557, 'liquidity': 308.36181823539135},
 'WETH001': {'LRNA': 71425.47037215637, 'liquidity': 1019.4586640314313},
 'ZTG': {'LRNA': 4538.957735895216, 'liquidity': 3133789.308258377},
 'iBTC': {'LRNA': 34932.3449207782

In [5]:

ob_objs = {}
for tkn_pair in order_books:
    ob = parse_kraken_orderbook(order_books[tkn_pair])
    ob_obj = OrderBook(
        bids=[[bid['price'], bid['amount']] for bid in ob['bids']],
        asks=[[ask['price'], ask['amount']] for ask in ob['asks']]
    )
    ob_objs[tkn_pair] = ob_obj
    
order_book_asset_list = []
for tkn_pair in order_books:
    for tkn in tkn_pair:
        if tkn not in order_book_asset_list:
            order_book_asset_list.append(tkn)

cex_fee = 0.0016
buffer = 0.0002

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}

# lrna_fee = 0.0005
# asset_fee = 0.0025


cex = CentralizedMarket(
    order_book=ob_objs,
    asset_list=order_book_asset_list,
    trade_fee=cex_fee
)

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)

init_state = op_state.copy()


In [6]:
order_book_map = {}
for i in range(len(arb_list)):
    base_id, quote_id = arb_list[i]['tkn_ids']
    orderbook_tkn_pair = arb_list[i]['order_book']
    if base_id in asset_map and quote_id in asset_map:
        tkn_pair = (asset_map[base_id], asset_map[quote_id])
        order_book_map[tkn_pair] = orderbook_tkn_pair 
    
pprint(order_book_map)

{('ASTR', 'DAI'): ('ASTR', 'USD'),
 ('ASTR', 'DAI001'): ('ASTR', 'USD'),
 ('ASTR', 'USDT'): ('ASTR', 'USD'),
 ('BNC', 'DAI'): ('BNC', 'USD'),
 ('BNC', 'DAI001'): ('BNC', 'USD'),
 ('BNC', 'USDT'): ('BNC', 'USD'),
 ('CFG', 'DAI'): ('CFG', 'USD'),
 ('CFG', 'DAI001'): ('CFG', 'USD'),
 ('CFG', 'USDT'): ('CFG', 'USD'),
 ('DOT', 'DAI'): ('DOT', 'USD'),
 ('DOT', 'DAI001'): ('DOT', 'USD'),
 ('DOT', 'USDT'): ('DOT', 'USD'),
 ('DOT', 'WBTC001'): ('DOT', 'XBT'),
 ('DOT', 'WETH'): ('DOT', 'ETH'),
 ('DOT', 'WETH001'): ('DOT', 'ETH'),
 ('DOT', 'iBTC'): ('DOT', 'XBT'),
 ('GLMR', 'DAI'): ('GLMR', 'USD'),
 ('GLMR', 'DAI001'): ('GLMR', 'USD'),
 ('GLMR', 'USDT'): ('GLMR', 'USD'),
 ('HDX', 'DAI'): ('HDX', 'USD'),
 ('HDX', 'DAI001'): ('HDX', 'USD'),
 ('HDX', 'USDT'): ('HDX', 'USD'),
 ('WBTC001', 'DAI'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'DAI001'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'USDT'): ('XXBT', 'ZUSD'),
 ('WETH', 'DAI'): ('XETH', 'ZUSD'),
 ('WETH', 'DAI001'): ('XETH', 'ZUSD'),
 ('WETH', 'USDT'): ('XETH', 'ZUS

In [7]:
all_swaps = get_arb_swaps(op_state, cex, order_book_map, buffer=buffer)
for tkn_pair in all_swaps:
    if all_swaps[tkn_pair] != []:
        print(tkn_pair)
        print(all_swaps[tkn_pair])

('DOT', 'DAI')
[{'dex': {'trade': 'sell', 'buy_asset': 'DAI', 'sell_asset': 'DOT', 'price': 4.121, 'amount': 8.307079109191896, 'min_buy': 34.29555296037197}, 'cex': {'trade': 'buy', 'buy_asset': 'DOT', 'sell_asset': 'USD', 'price': 4.121412100000001, 'amount': 8.307079109191896}}]
('WETH', 'USDT')
[{'dex': {'trade': 'buy', 'buy_asset': 'WETH', 'sell_asset': 'USDT', 'price': 1775.49, 'amount': 0.081, 'max_sell': 143.3352796226382}, 'cex': {'trade': 'sell', 'buy_asset': 'ZUSD', 'sell_asset': 'XETH', 'price': 1775.312451, 'amount': 0.081}}, {'dex': {'trade': 'buy', 'buy_asset': 'WETH', 'sell_asset': 'USDT', 'price': 1775.11, 'amount': 0.13432408332824708, 'max_sell': 381.2398680690765}, 'cex': {'trade': 'sell', 'buy_asset': 'ZUSD', 'sell_asset': 'XETH', 'price': 1774.932489, 'amount': 0.13432408332824708}}]
('WETH', 'DAI001')
[{'dex': {'trade': 'buy', 'buy_asset': 'WETH', 'sell_asset': 'DAI001', 'price': 1775.49, 'amount': 0.081, 'max_sell': 143.39712514868975}, 'cex': {'trade': 'sell', 

In [8]:
state = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list + order_book_asset_list}
init_agent = Agent(holdings=holdings, unique_id='bot')
agent = init_agent.copy()
execute_arb(state, cex, agent, all_swaps)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'WETH001': 'ETH',
             'IBTC': 'BTC',
             'XBT': 'BTC',}

profit = calculate_profit(init_agent, agent, asset_map)

print(profit)
            

{'HDX': 0, 'USD': 8.508431315422058, 'BTC': 0.08130598068237305, 'ETH': 0.0, 'DOT': 0.0, 'ASTR': 0, 'iBTC': -0.08121263980865479, 'ZTG': 0, 'CFG': 0.0, 'BNC': 0, 'vDOT': 0, 'GLMR': 0}
